<a href="https://colab.research.google.com/github/t8101349/Colab-/blob/master/%E7%88%AC%E8%9F%B2homework_costco%E5%84%AA%E6%83%A0%E6%9F%A5%E8%A9%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import os
import json
import pandas as pd
from bs4 import BeautifulSoup
import time
import re


In [ ]:
# API URL
api_url = "https://www.costco.com.tw/rest/v2/taiwan/products/search"


# 添加必要的請求參數和標頭
params = {
    "fields": "FULL",
    "query": ":BazaarVoiceRating-desc:bazaarVoiceAverageRatingFacet:4-5:bazaarVoiceAverageRatingFacet:5:bazaarVoiceAverageRatingFacet:3-4",
    "pageSize": 48,
    "sort": "BazaarVoiceRating-desc",
    "category": "Coupon",
    "lang": "zh_TW",
    "curr": "TWD",
    "currentPage": 0
}  # 3星以上的特價商品



def download_img(url, save_path):
    response = requests.get(url)

    with open(save_path, "wb") as file:
        file.write(response.content)



def fetch_data(url,params):
    global bargain_list
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
        "Accept": "application/json, text/plain, */*"  # 確保接收 JSON 格式
    }
    # 發送 GET 請求
    response = requests.get(url, headers=headers, params=params)

    # 確認請求成功
    if response.status_code == 200:

        result = json.loads(response.text) # 解析 JSON 資料
        products = result['products']
        print(products)

        dir_name = "Costco_products_img"  # 建立資料夾
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)

        for product in products:  #商品資料
            name = product.get("name")
            price = product.get("price", {'value':"售完"}).get("value")
            couponDiscount = product.get("couponDiscount", {'discountValue':"N/A"}).get("discountValue","N/A")
            currency = product.get("price", {}).get("currencyIso")
            rating = product.get("averageRating", "無評價")

            # 抓取圖片 URL
            images = product.get("images", [])
            image_url = images[0].get("url") if images else "無圖片"

            # 列印檢查
            print(f"商品名稱: {name}")
            print(f"價格: {price} {currency}")
            print(f"優惠價: {couponDiscount}")
            print(f"評價分數: {rating}")
            print(f"圖片 URL: {image_url}\n")


            #下載圖片
            if image_url != "無圖片":
              image_url = 'https://www.costco.com.tw'+image_url
              f_name = re.sub(r'[\\/*?:"<>|]', "", name) + ".jpg"
              f_path = os.path.join(dir_name, f_name)
              download_img(image_url, f_path)  #下載圖片


            bargain_data = [name,price,couponDiscount,rating,image_url ]
            bargain_list.append(bargain_data)


        #自動換頁
        if params["currentPage"] >= result['pagination']["totalPages"] - 1:
            #輸出excel
            bargain_df = pd.DataFrame(bargain_list, columns = ["Name",f"Price({currency})", "Discount", 'Rating','Image'])
            bargain_df.to_csv('Costco on-sale.csv', index = False)
            print(bargain_list)

        else:
            params["currentPage"] += 1
            time.sleep(2)
            print(params["currentPage"])
            fetch_data(url,params)




    else:
        print(f"請求失敗，狀態碼: {response.status_code}")

if __name__ == "__main__":
    bargain_list = []
    fetch_data(api_url,params)

[{'addToCartFromPLPCategories': [{'key': 'hero-texture-of-life', 'value': True}, {'key': 'hot-buys', 'value': True}, {'key': 'hero-promotion', 'value': False}, {'key': 'Hero_Bed', 'value': False}, {'key': 'hero-onlineexclusive', 'value': True}, {'key': 'Hero_ChristmasDecorat', 'value': True}, {'key': 'FINLANDIA', 'value': True}, {'key': 'decal-position-1-OnlineExclusive', 'value': False}, {'key': 'hero-99015', 'value': False}, {'key': 'hero_allforu', 'value': True}, {'key': 'Hero_indoor', 'value': True}, {'key': 'Hero_Homegift', 'value': True}, {'key': 'Hero_LivingRoom', 'value': True}, {'key': 'decal-position-1-IRC', 'value': False}, {'key': 'Hero_IndoorFurniture', 'value': True}, {'key': 'Hero_Furniture', 'value': True}, {'key': 'hero-Cost-effective', 'value': True}, {'key': 'hero-sleepy', 'value': False}, {'key': 'Coupon', 'value': True}, {'key': 'brands', 'value': False}, {'key': 'hero-lifestyleofautumn', 'value': True}, {'key': 'CL5', 'value': True}, {'key': 'hero-Bedroom', 'value

KeyboardInterrupt: 